Adopted from work by Steve Phelps:
https://github.com/phelps-sg/python-bigdata and Dr. Zhu, Yu
This work is licensed under the Creative Commons Attribution 4.0 International license agreement.



### Overview

1. Recap of functional programming in Python
2. Python's `map` and `reduce` functions
3. Writing parallel code using `map`
4. The Map-Reduce programming model



## History

- The Map-Reduce programming model was popularised by Google (Dean and Ghemawat 2008).

- The first popular open-source implementation was Apache Hadoop, first released in 2011.


## Functional programming

Consider the following code:

In [1]:
def double_everything_in(data):
    result = []
    for i in data:
        result.append(2 * i)
    return result

def quadruple_everything_in(data):
    result = []
    for i in data:
        result.append(4 * i)
    return result

In [7]:
mylist = [1,2,3,4,5,5]

In [8]:
double_everything_in(mylist)

[2, 4, 6, 8, 10, 10]

In [10]:
quadruple_everything_in(mylist)

[4, 8, 12, 16, 20, 20]

In [12]:
def multiply_by_x_everything_in(x, data):
    result = []
    for i in data:
        result.append(x * i)
    return result

In [14]:
multiply_by_x_everything_in(0.5, mylist)

[0.5, 1.0, 1.5, 2.0, 2.5, 2.5]

In [15]:
multiply_by_x_everything_in(4, mylist)

[4, 8, 12, 16, 20, 20]

- Now consider the following code:

In [16]:
def squared(x):
    return x*x

def double(x):
    return x*2

def square_everything_in(data):
    result = []
    for i in data:
        result.append(squared(i))
    return result

def double_everything_in(data):
    result = []
    for i in data:
        result.append(double(i))
    return result

In [17]:
square_everything_in([1, 2, 3, 4, 5])

[1, 4, 9, 16, 25]

In [18]:
double_everything_in([1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

### DRY - Fundamental Programming Concept
- The above code violates the ["do not repeat yourself"](https://en.wikipedia.org/wiki/Don't_repeat_yourself_) principle of good software engineering practice.

- How can rewrite the code so that it avoids duplication?

### Passing Functions as Values
- Functions can be passed to other functions as values.


In [19]:
def apply_f_to_everything_in(f, data):
    result = []
    for x in data:
        result.append(f(x))
    return result

In [20]:
apply_f_to_everything_in(squared, [1, 2, 3, 4, 5])

[1, 4, 9, 16, 25]

In [21]:
apply_f_to_everything_in(double, [1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

In [32]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

# Applying factorial function to the data list
factorial_results = apply_f_to_everything_in(factorial, mylist)

print(factorial_results)


[1, 2, 6, 24, 120, 120]


### Lambda expressions

- We can use anonymous functions to save having to define a function each time we want to use map.

In [ ]:
apply_f_to_everything_in(lambda x: x*x, [1, 2, 3, 4, 5])

[1, 4, 9, 16, 25]

# Python's `map` function

- Python has a built-in function `map` which is much faster than our version.



In [33]:
map(lambda x: x*x, [1, 2, 3, 4, 5])

## Implementing reduce

- The `reduce` function is an example of a [fold](https://en.wikipedia.org/wiki/Fold_%28higher-order_function%29).

- There are different ways we can fold data.

- The following implements a *left* fold.


In [94]:
def foldl(f, data, z):
    if (len(data) == 0):
        print (z)
        return z
    else:
        head = data[0]
        tail = data[1:]
        print ("Folding", head, "with", tail, "using", z)
        partial_result = f(z, data[0])
        print ("Partial result is", partial_result)
        return foldl(f, tail, partial_result)

 1. A left fold iteratively applies a function f to an accumulator z and the elements of the list data from left to right.
 2. partial_result = f(z, head): Apply the function f to the current accumulator z and the first element head of data, and store the result in partial_result

In [95]:
def add(x, y):
    return x + y

foldl(add, [3, 3, 3, 3, 3], 0)

Folding 3 with [3, 3, 3, 3] using 0
Partial result is 3
Folding 3 with [3, 3, 3] using 3
Partial result is 6
Folding 3 with [3, 3] using 6
Partial result is 9
Folding 3 with [3] using 9
Partial result is 12
Folding 3 with [] using 12
Partial result is 15
15


15

In [40]:
foldl(lambda x, y: x + y, [1, 2, 3, 4, 5], 0)

Folding 1 with [2, 3, 4, 5] using 0
Partial result is 1
Folding 2 with [3, 4, 5] using 1
Partial result is 3
Folding 3 with [4, 5] using 3
Partial result is 6
Folding 4 with [5] using 6
Partial result is 10
Folding 5 with [] using 10
Partial result is 15
15


15

In [97]:
foldl(lambda x, y: x - y, [1, 2, 3, 4, 5], 5)

Folding 1 with [2, 3, 4, 5] using 5
Partial result is 4
Folding 2 with [3, 4, 5] using 4
Partial result is 2
Folding 3 with [4, 5] using 2
Partial result is -1
Folding 4 with [5] using -1
Partial result is -5
Folding 5 with [] using -5
Partial result is -10
-10


-10

### left fold is efficient when summing up things

In [101]:
def concatenate(x, y):
    return x + y

data = ["a", "b", "c", "d"]
concat_result = foldl(concatenate, data, "")  # Output: "abcd"


Folding a with ['b', 'c', 'd'] using !
Partial result is !a
Folding b with ['c', 'd'] using !a
Partial result is !ab
Folding c with ['d'] using !ab
Partial result is !abc
Folding d with [] using !abc
Partial result is !abcd
!abcd


In [102]:
(((((0 - 1) - 2) - 3) - 4) - 5)

-15

In [103]:
def subtract(x, y):
    return x - y

# Example data
data = [1, 2, 3, 4, 5]
initial_value = 0

# Applying subtract function to the data list using foldl
result = foldl(subtract, data, initial_value)

print("Final result:", result)

Folding 1 with [2, 3, 4, 5] using 0
Partial result is -1
Folding 2 with [3, 4, 5] using -1
Partial result is -3
Folding 3 with [4, 5] using -3
Partial result is -6
Folding 4 with [5] using -6
Partial result is -10
Folding 5 with [] using -10
Partial result is -15
-15
Final result: -15


- Subtraction is neither [commutative](https://en.wikipedia.org/wiki/Commutative_property) nor [associative](https://en.wikipedia.org/wiki/Associative_property), so the order in which apply the fold matters:

### Right Fold
* This can be particularly useful in scenarios where the operation naturally associates to the right or when you need to preserve the structure of the list during the fold.

In [104]:
(1 - (2 - (3 - (4 - (5 - 0)))))

3

(1 - (2 - (3 - (4 - (5 - 0)))))

* Starting from the innermost parentheses:
(5 - 0) = 5
* Next, we have:
(4 - 5) = -1
* Then, we have:
(3 - (-1)) = 4
* Next, we have:
(2 - 4) = -2
* Finally, we have:
(1 - (-2)) = 3

In [105]:
def foldr(f, data, z):
    if (len(data) == 0):
        return z# define accumulator
    else:
        return f(data[0], foldr(f, data[1:], z))

In [106]:
foldr(lambda x, y: x - y,  [1, 2, 3, 4, 5], 0)

3

In [46]:
foldl(lambda x, y: x - y,  [1, 2, 3, 4, 5], 0)

Folding 1 with [2, 3, 4, 5] using 0
Partial result is -1
Folding 2 with [3, 4, 5] using -1
Partial result is -3
Folding 3 with [4, 5] using -3
Partial result is -6
Folding 4 with [5] using -6
Partial result is -10
Folding 5 with [] using -10
Partial result is -15
-15


-15

(1−(2−(3−(4−(5−0)))))
we subtract each element of the list from the accumulator starting from the rightmost element (5) and moving towards the leftmost element (1).
* Accumulator: 0−5=−5
* Accumulator: −5-4=−9
* ...
* -15

In [77]:
foldr(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

3

The function call is evaluated as follows:
* foldr(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)
* (1 - foldr(lambda x, y: x - y, [2, 3, 4, 5], 0))
* (1 - (2 - foldr(lambda x, y: x - y, [3, 4, 5], 0)))
* (1 - (2 - (3 - foldr(lambda x, y: x - y, [4, 5], 0))))
* (1 - (2 - (3 - (4 - foldr(lambda x, y: x - y, [5], 0)))))
* (1 - (2 - (3 - (4 - (5 - 0)))))
* The final result is 3.


#### The foldr function is used to perform a series of subtractions on the elements of the listS, starting with the initial value 0.

## Python's `reduce` function.

- Python's built-in `reduce` function is a *left* fold.

In [54]:
from functools import reduce
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])

15

In [55]:
reduce(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

-15

In [56]:
foldl(lambda x, y: x - y, [1, 2, 3, 4, 5], 0)

Folding 1 with [2, 3, 4, 5] using 0
Partial result is -1
Folding 2 with [3, 4, 5] using -1
Partial result is -3
Folding 3 with [4, 5] using -3
Partial result is -6
Folding 4 with [5] using -6
Partial result is -10
Folding 5 with [] using -10
Partial result is -15
-15


-15

# Functional programming and parallelism

- Functional programming lends itself to [parallel programming](https://computing.llnl.gov/tutorials/parallel_comp/#Models).

- The `map` function can easily be parallelised through [data-level parallelism](https://en.wikipedia.org/wiki/Data_parallelism),
    - provided that the function we supply as an argument is *free from* [side-effects](https://en.wikipedia.org/wiki/Side_effect_%28computer_science%29)
        - (which is why we avoid working with mutable data).

- We can see this by rewriting it so:


In [107]:
def perform_computation(f, result, data, i):
    print ("Computing the ", i, "th result...")
    # This could be scheduled on a different CPU
    result[i] = f(data[i])

def my_map(f, data):
    result = [None] * len(data)
    for i in range(len(data)):
        perform_computation(f, result, data, i)
    # Wait for other CPUs to finish, and then..
    return result

#### my_map Function:

* This function simulates a parallel map operation. It takes a function f and a list data.
* It initializes a result list (result) with the same length as data, filled with None.
* It iterates over each element of data, and for each element, it calls perform_computation to compute the result for that element in parallel.
* After all computations are done (simulated by the loop), it returns the result list containing the computed results.

Summary:
* The perform_computation() function is responsible for applying a given function to a single element of the input data.
* The my_map() function orchestrates the parallel execution of these computations and collects the results.

In [109]:
my_map(lambda x: x + x, [1, 2, 3, 4, 5])

Computing the  0 th result...
Computing the  1 th result...
Computing the  2 th result...
Computing the  3 th result...
Computing the  4 th result...


[2, 4, 6, 8, 10]

## A multi-threaded `map` function

In [110]:
from threading import Thread

def schedule_computation_threaded(f, result, data, threads, i):
    # Each function evaluation is scheduled on a different core.
    def my_job():
        print ("Processing data:", data[i], "... ")
        result[i] = f(data[i])
        print ("Finished job #", i)
        print ("Result was", result[i])
    threads[i] = Thread(target=my_job)

def my_map_multithreaded(f, data):
    n = len(data)
    result = [None] * n
    threads = [None] * n
    print ("Scheduling jobs.. ")
    for i in range(n):
        schedule_computation_threaded(f, result, data, threads, i)
    print ("Starting jobs.. ")
    for i in range(n):
        threads[i].start()
    print ("Waiting for jobs to finish.. ")
    for i in range(n):
        threads[i].join()
    print ("All done.")
    return result

* from threading import Thread:
This line imports the Thread class from the threading module, which will be used to create and manage threads for the parallelized computation.

* def schedule_computation_threaded(f, result, data, threads, i):
This function is responsible for scheduling a single computation on a separate thread.
It takes the following arguments:
1. f: the function to be applied to the current element
2. result: the list to store the results
3. data: the input data list
threads: a list to store the created threads
4. i: the index of the current element being processed

It defines a nested function my_job(), which is the actual computation that will be performed on a separate thread.

* Inside my_job(), it prints a message indicating that it is processing the current data element, applies the function f to the current data element, and stores the result in the result list at index i.
* It then creates a new Thread object, assigns the my_job() function as the target, and stores the thread in the threads list at index i.

* def my_map_multithreaded(f, data):
This function is similar to the previous my_map() function, but it uses threads to parallelize the computations.
1. It first initializes the result list and the threads list, both of length n, where n is the length of the data list.
2. It then prints a message indicating that it is scheduling the jobs.
3. It calls the schedule_computation_threaded() function for each element of the data list, passing in the necessary arguments.
4. After scheduling all the jobs, it prints a message indicating that it is starting the jobs.
5. It then starts each of the threads in the threads list by calling the start() method.
6. It prints a message indicating that it is waiting for the jobs to finish.
7. It then joins all the threads in the threads list by calling the join() method on each thread, effectively waiting for all the computations to complete.
8. Finally, it prints a message indicating that all the jobs are done and returns the result list.
* The key difference between this code and the previous one is the use of threads to parallelize the computations. Instead of scheduling the computations on different CPUs (as mentioned in the previous code), this code creates a separate thread for each computation and manages them using the threading module.

* This approach can potentially improve the performance of the computations, especially if the underlying hardware has multiple CPU cores that can run the threads concurrently. However, the use of threads also introduces some additional complexity, such as the need to manage thread synchronization and potential issues like race conditions or deadlocks.

In [111]:
my_map_multithreaded(lambda x: x*x, [1, 2, 3, 4, 5])

Scheduling jobs.. 
Starting jobs.. 
Processing data:Processing data: 2 ... 
Finished job # 1
Result was 4
 1 ... 
Finished job # 0
Result was 1
Processing data: 3 ... 
Finished job # 2
Result was 9
Processing data: 4 ... 
Finished job # 3
Result was Processing data: 5Waiting for jobs to finish.. 
 ... 
Finished job # 4
Result was 25
16
All done.


[1, 4, 9, 16, 25]

In [79]:
from numpy.random import uniform
from time import sleep

def a_function_which_takes_a_long_time(x):
    sleep(uniform(2, 10))  # Simulate some long computation
    return x*x

my_map_multithreaded(a_function_which_takes_a_long_time, [1, 2, 3, 4, 5])

Scheduling jobs.. 
Starting jobs.. 
Processing data: 1 ... 
Processing data: 2 ... 
Processing data: 3 ... 
Processing data:Processing data:Waiting for jobs to finish.. 
 5 ... 
 4 ... 
Finished job # 4
Result was 25
Finished job # 2
Result was 9
Finished job # 3
Result was 16
Finished job # 1
Result was 4
Finished job # 0
Result was 1
All done.


[1, 4, 9, 16, 25]

## Map Reduce

- Map Reduce is a _programming model_ for scalable parallel processing.
- Scalable here means that it can work on big data with very large compute clusters.
- There are many implementations: e.g. Apache Hadoop and Apache Spark.
- We can use Map-Reduce with any programming language:
    - Hadoop is written in Java
    - Spark is written in Scala, but has a Python interface.
- *Functional programming* languages such as Python or Scala fit very well with the Map Reduce model:
    - However, we don't *have* to use functional programming.

- A MapReduce implementation will take care of the low-level functionality so that you don't have to worry about:
    - load balancing
    - network I/O
    - network and disk transfer optimization
    - handling of machine failures
    - serialization of data
    - etc..
- The model is designed to move the processing to where the data resides.

## Typical steps in a Map Reduce Computation

1. ETL a big data set.
2. _Map_ operation: extract something you care about from each row
3. "Shuffle and Sort": task/node allocation
4. _Reduce_ operation: aggregate, summarise, filter or transform
5. Write the results.

## Callbacks for Map Reduce

- The data set, and the state of each stage of the computation, is represented as a set of key-value pairs.

- The programmer provides a map function:

$\operatorname{map}(k, v) \rightarrow \; \left< k', v' \right>*$  

- and a reduce function:

$\operatorname{reduce}(k', \left< k', v'\right> *) \rightarrow \; \left< k', v''
\right> *$

- The $*$ refers to a *collection* of values.

- These collections are *not* ordered.

## Word Count Example

- In this simple example, the input is a set of URLs, each record is a document.

- Problem: compute how many times each word has occurred across data set.

## Word Count: Map


- The input to $\operatorname{map}$ is a mapping:

- Key: URL
- Value: Contents of document

$\left< document1, to \; be \; or \; not \; to \; be \right>$  
    

- In this example, our $\operatorname{map}$ function will process a given URL, and produces a mapping:

- Key: word
- Value: 1

- So our original data-set will be transformed to:
  
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$
  $\left< or, 1 \right>$
  $\left< not, 1 \right>$
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$

## Word Count: Reduce


- The reduce operation groups values according to their key, and then performs a reduce on each key.

- The collections are partitioned across different storage units, therefore.

- Map-Reduce will fold the data in such a way that it minimises data-copying across the cluster.

- Data in different partitions are reduced separately in parallel.

- The final result is a reduce of the reduced data in each partition.

- Therefore it is very important that our operator *is both commutative and associative*.

- In our case the function is the `+` operator

  $\left< be, 2 \right>$  
  $\left< not, 1 \right>$  
  $\left< or, 1 \right>$  
  $\left< to, 2 \right>$  
  

## Map and Reduce compared with Python

- Notice that these functions are formulated differently from the standard Python functions of the same name.

- The `reduce` function works with key-value *pairs*.

- It would be more apt to call it something like `reduceByKey`.


## MiniMapReduce

- To illustrate how the Map-Reduce programming model works, we can implement our own Map-Reduce framework in Python.

- This *illustrates* how a problem can be written in terms of `map` and `reduce` operations.

- Note that these are illustrative functions; this is *not* how Hadoop or Apache Spark actually implement them.

In [87]:
##########################################################
#
#   MiniMapReduce
#
# A non-parallel, non-scalable Map-Reduce implementation
##########################################################

def groupByKey(data):
    result = dict()
    for key, value in data:
        if key in result:
            result[key].append(value)
        else:
            result[key] = [value]
    return result

def reduceByKey(f, data):
    key_values = groupByKey(data)
    return map(lambda key:
                   (key, reduce(f, key_values[key])),
                       key_values)

## Word-count using MiniMapReduce


In [88]:
data = map(lambda x: (x, 1), "to be or not to be".split())
data

#### Returned a map object

In [89]:
groupByKey(data)

{'to': [1, 1], 'be': [1, 1], 'or': [1], 'not': [1]}

In [90]:
reduceByKey(lambda x, y: x + y, data)

## Parallelising MiniMapReduce

- We can easily turn our Map-Reduce implementation into a parallel, multi-threaded framework
by using the `my_map_multithreaded` function we defined earlier.

- This will allow us to perform map-reduce computations that exploit parallel processing using *multiple* cores on a *single* computer.

In [91]:
def reduceByKey_multithreaded(f, data):
    key_values = groupByKey(data)
    return my_map_multithreaded(
        lambda key: (key, reduce(f, key_values[key])), key_values.keys())

In [92]:
reduceByKey_multithreaded(lambda x, y: x + y, data)

Scheduling jobs.. 
Starting jobs.. 
Waiting for jobs to finish.. 
All done.


[]

## Parallelising the reduce step

- Provided that our operator is both associative and commutative we can
also parallelise the reduce operation.

- We partition the data into approximately equal subsets.

- We then reduce each subset independently on a separate core.

- The results can be combined in a final reduce step.

### Partitioning the data

In [93]:
def split_data(data, split_points):
    partitions = []
    n = 0
    for i in split_points:
        partitions.append(data[n:i])
        n = i
    partitions.append(data[n:])
    return partitions

data = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
partitioned_data = split_data(data, [3])
partitioned_data

[['a', 'b', 'c'], ['d', 'e', 'f', 'g']]

### Reducing across partitions in parallel

In [76]:
from threading import Thread

def parallel_reduce(f, partitions):

    n = len(partitions)
    results = [None] * n # to store the result
    threads = [None] * n # to store the object

    def job(i):
        results[i] = reduce(f, partitions[i])
        # this aims to execute job() on each partition

    for i in range(n):
        threads[i] = Thread(target = lambda: job(i))
        threads[i].start()
        # creates a new Thread object, where the target argument
        # specifies the function that the thread will execute the job() when it starts

    for i in range(n):
        threads[i].join()

    return reduce(f, results)

parallel_reduce(lambda x, y: x + y, partitioned_data)

'abcdefg'

## Map-Reduce on a cluster of computers

- The code we have written so far will *not* allow us to exploit parallelism from multiple computers in a [cluster](https://en.wikipedia.org/wiki/Computer_cluster).

- Developing such a framework would be a very large software engineering project.

- There are existing frameworks we can use:
    - [Apache Hadoop](https://hadoop.apache.org/)
    - [Apache Spark](https://spark.apache.org/)
    
- In the next notebook we will cover Apache Spark.